In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import lines as mlines
from astropy.cosmology import Planck13

In [2]:
import os
from astropy.table import Table
dirname = "/Users/aphearin/Dropbox/UniverseMachine/data/histories/prelim_sfh_reduction"
history_fname = "sfh_catalog_1.002310_smcut_9.50_value_added_histories.hdf5"
histories = Table.read(os.path.join(dirname, history_fname), path='data')
cosmic_age_array = np.load(os.path.join(dirname, 'times.npy'))

In [3]:
from read_sfh import get_scales
dirname="/Users/aphearin/Dropbox/UniverseMachine/data/histories"
basename="small_sfh_catalog_1.002310.txt"
tree_fname = os.path.join(dirname, basename)
scale_factor_array = get_scales(tree_fname)
redshift_array = (1./scale_factor_array) - 1
redshift_array[-1] = 0.

## Cross-match to get Vmax at Mpeak

In [9]:
from halotools.sim_manager import CachedHaloCatalog
halocat = CachedHaloCatalog(simname='bolplanck')

from halotools.utils import crossmatch
idxA, idxB = crossmatch(histories['id'], halocat.halo_table['halo_id'])

histories['vmax_at_mpeak'] = 0.
histories['vmax_at_mpeak'][idxA] = halocat.halo_table['halo_vmax_mpeak'][idxB]

### Build a lookup table

In [15]:
from halotools.mock_observables import mean_y_vs_x

log_vmax_table, log_sm_table, __ = mean_y_vs_x(np.log10(histories['vmax_at_mpeak']), 
                                           np.log10(histories['sm']))

## Calculate evolution of main sequence peak for different $M_*$

In [17]:
from main_sequence_sfr import mean_sfr
from sfr_designation import quiescent_fraction